In [1]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = 'ae810dc4282e3de074ab08ab631f3461218accf99143b8f3a272e64c1b4c6d0e'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [2]:
# 未玩新地图的用户的付费
df_pay_1 = ssql("""
SELECT t2.pay
FROM
  (SELECT t1.distinct_id
   FROM
     (SELECT date,distinct_id
      FROM events
      WHERE gameTypeId = 1800
        AND event = 'gameOver'
        AND gameSubType = '4'
        AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
      GROUP BY 1,
               2)t1
   LEFT JOIN
     (SELECT date,distinct_id
      FROM events
      WHERE gameTypeId = 1800
        AND event = 'gameOver'
        AND gameSubType = '4'
        AND detail = 'temple'
        AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
      GROUP BY 1,
               2)t2 ON t1.date = t2.date
   AND t1.distinct_id = t2.distinct_id
   WHERE t2.distinct_id IS NULL
   GROUP BY 1) t1
JOIN
  (SELECT distinct_id,
          sum(pay)AS pay
   FROM events
   WHERE event = 'pay'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
""")

In [26]:
pd.DataFrame(df_pay_1['pay'].quantile([0.1,0.25,0.5,0.75,0.9])).T.to_clipboard()

In [3]:
# 玩过新地图的用户付费
df_pay_2 = ssql("""

SELECT t2.pay
FROM
  (SELECT distinct_id
   FROM events
   WHERE gameTypeId = 1800
     AND event = 'gameOver'
     AND gameSubType = '4'
     AND detail = 'temple'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t1
JOIN
  (SELECT distinct_id,
          sum(pay)AS pay
   FROM events
   WHERE event = 'pay'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
""")

In [27]:
pd.DataFrame(df_pay_2['pay'].quantile([0.1,0.25,0.5,0.75,0.9])).T.to_clipboard()

In [4]:
# 玩过新地图用户的分场局数
df_sub_1 = ssqlql("""

SELECT t2.distinct_id,
       max(CASE WHEN t2.gameSubType = '1' THEN sub_num ELSE 0 END) `新手场局数`,
       max(CASE WHEN t2.gameSubType = '2' THEN sub_num ELSE 0 END) `躲猫猫局数`,
       max(CASE WHEN t2.gameSubType = '3' THEN sub_num ELSE 0 END) `进阶场局数`,
       max(CASE WHEN t2.gameSubType = '4' THEN sub_num ELSE 0 END) `12人3狼场局数`,
       max(CASE WHEN t2.gameSubType = '5' THEN sub_num ELSE 0 END) `躲猫猫欢乐技能场`
FROM
  (SELECT distinct_id
   FROM events
   WHERE gameTypeId = 1800
     AND event = 'gameOver'
     AND gameSubType = '4'
     AND detail = 'temple'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t1
JOIN
  (SELECT distinct_id,
          gameSubType,
          count(distinct_id)AS sub_num
   FROM events
   WHERE gameTypeId = 1800
     AND event = 'gameOver'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
GROUP BY 1
""")

In [28]:
pd.DataFrame(df_sub_1[['进阶场局数','12人3狼场局数','躲猫猫欢乐技能场','新手场局数','躲猫猫局数']].quantile([0.1,0.25,0.5,0.75,0.9])).T.to_clipboard()

In [5]:
# 未玩过新地图的用户的分场局数
df_sub_2 = ssql("""

SELECT t2.distinct_id,
       max(CASE WHEN t2.gameSubType = '1' THEN sub_num ELSE 0 END) `新手场局数`,
       max(CASE WHEN t2.gameSubType = '2' THEN sub_num ELSE 0 END) `躲猫猫局数`,
       max(CASE WHEN t2.gameSubType = '3' THEN sub_num ELSE 0 END) `进阶场局数`,
       max(CASE WHEN t2.gameSubType = '4' THEN sub_num ELSE 0 END) `12人3狼场局数`,
       max(CASE WHEN t2.gameSubType = '5' THEN sub_num ELSE 0 END) `躲猫猫欢乐技能场`
FROM
  (SELECT t1.distinct_id
   FROM
     (SELECT date,distinct_id
      FROM events
      WHERE gameTypeId = 1800
        AND event = 'gameOver'
        AND gameSubType = '4'
        AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
      GROUP BY 1,
               2)t1
   LEFT JOIN
     (SELECT date,distinct_id
      FROM events
      WHERE gameTypeId = 1800
        AND event = 'gameOver'
        AND gameSubType = '4'
        AND detail = 'temple'
        AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
      GROUP BY 1,
               2)t2 ON t1.date = t2.date
   AND t1.distinct_id = t2.distinct_id
   WHERE t2.distinct_id IS NULL
   GROUP BY 1) t1
JOIN
  (SELECT distinct_id,
          gameSubType,
          count(distinct_id)AS sub_num
   FROM events
   WHERE gameTypeId = 1800
     AND event = 'gameOver'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
GROUP BY 1
""")

In [29]:
pd.DataFrame(df_sub_2[['进阶场局数','12人3狼场局数','躲猫猫欢乐技能场','新手场局数','躲猫猫局数']].quantile([0.1,0.25,0.5,0.75,0.9])).T.to_clipboard()

In [6]:
# 玩过新地图的用户加好友次数
df_friend_1 = ssql("""

SELECT t2.friend_num
FROM
  (SELECT distinct_id
   FROM events
   WHERE gameTypeId = 1800
     AND event = 'gameOver'
     AND gameSubType = '4'
     AND detail = 'temple'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t1
JOIN
  (SELECT distinct_id,
          count(distinct_id) friend_num
   FROM events
   WHERE event = 'addFriendNew'
     AND RESULT = '0'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
""")

In [30]:
pd.DataFrame(df_friend_1['friend_num'].quantile([0.1,0.25,0.5,0.75,0.9])).T.to_clipboard()

In [8]:
# 未玩过新地图的用户的加好友次数
df_friend_2 = ssql("""

SELECT t2.friend_num
FROM
  (SELECT t1.distinct_id
   FROM
     (SELECT date,distinct_id
      FROM events
      WHERE gameTypeId = 1800
        AND event = 'gameOver'
        AND gameSubType = '4'
        AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
      GROUP BY 1,
               2)t1
   LEFT JOIN
     (SELECT date,distinct_id
      FROM events
      WHERE gameTypeId = 1800
        AND event = 'gameOver'
        AND gameSubType = '4'
        AND detail = 'temple'
        AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
      GROUP BY 1,
               2)t2 ON t1.date = t2.date
   AND t1.distinct_id = t2.distinct_id
   WHERE t2.distinct_id IS NULL
   GROUP BY 1) t1
JOIN
  (SELECT distinct_id,
          count(distinct_id) friend_num
   FROM events
   WHERE event = 'addFriendNew'
     AND RESULT = '0'
     AND date BETWEEN '2021-02-02' AND current_date() - interval 1 DAY
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
""")

In [31]:
pd.DataFrame(df_friend_2['friend_num'].quantile([0.1,0.25,0.5,0.75,0.9])).T.to_clipboard()